## Wind power prediction
### 流程
1. 先從 SQL 下載 WF01 的風機資料從 20210801~20220731
2. 製造出剛好在時間間隔上的時間戳記，並補足缺的時間點，和刪除重複或小於時間間隔的資料
3. 針對不同 DCS#LINE and DCS#TAGCD 切出 116 個 features
4. 製造 Label，和先刪除對 label 不合理的資料
5. 使用移動平均補通訊異常值
6. 對 label 做 時間序列檢定，和畫 ACF and PACF 決定下一步要製造 lag幾期的 features
7. 製造出 t-1 ~ t-6，的 features 並移除除了 t 時點的 features
8. 選取和 label 相關係數前 20 高的當成 model 的 features
9. 針對上一步所選取的 features 做 EDA
10. 隨機切分 train set and test set, test_size= 0.2
11. 訓練模型(XGBoost, Random Forest, SVM, Ridge, Bayian regression)
12. hyperparameter fintuning
13. Emndeeing Method

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pymssql
import datetime 
from datetime import timedelta
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import statsmodels.tsa as ts
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, Ridge, BayesianRidge
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer, r2_score
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, StackingRegressor, BaggingRegressor
from scipy.stats import uniform, randint
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### 從 SQL 下載 WF01 的風機資料從 20210801~20220731

In [ ]:
conn = pymssql.connect(
    host= '192.168.204.14',
    user = 'sa',
    password = '!qaz2wsx',
    database = 'TGEC_OPC_DATA_2021'
)
cursor = conn.cursor()

In [ ]:
columns = ['DCS#PLANT', 'DCS#LINE', 'DCS#TAGCD', 'DCS#VALUE', 'DCS#DATETIME', 'YMD', 'HH', 'MI']
sql = "SELECT * from  DCS_DATA WHERE DCS#LINE like 'WF01%' AND YMD>= 20210801 AND YMD<=20211231"
cursor.execute(sql)
row = cursor.fetchall()
data_2021 = pd.DataFrame(row, columns= columns)

In [ ]:
conn = pymssql.connect(
    host= '192.168.204.14',
    user = 'sa',
    password = '!qaz2wsx',
    database = 'TGEC_OPC_DATA_History'
)
cursor = conn.cursor()

In [ ]:
columns = ['DCS#PLANT', 'DCS#LINE', 'DCS#TAGCD', 'DCS#VALUE', 'DCS#DATETIME', 'YMD', 'HH', 'MI']
sql = "SELECT * from  DCS_DATA WHERE DCS#LINE like 'WF01%' AND YMD>= 20220101 AND YMD<= 20220731"
cursor.execute(sql)
row = cursor.fetchall()
data_2022 = pd.DataFrame(row, columns= columns)

### 製造出沒有秒數的時間戳記
原始的時間可能會多個幾秒，在對時間比較麻煩，但 raw data 有提供 剛好在時間間隔上的時間，不過 type 是文字，將其合併轉成時間類型(datetime)，取代原始 DCS#DATETIME

In [ ]:
ALL = pd.concat([data_2021, data_2022], axis= 0)
ALL = ALL.sort_values(by= ['DCS#LINE', 'DCS#TAGCD', 'DCS#DATETIME'])
ALL['datetime'] = pd.to_datetime(ALL['YMD'] + ALL['HH'] + ALL['MI'], format= '%Y%m%d%H%M')
ALL['DCS#VALUE'] = ALL['DCS#VALUE'].astype(float)
ALL.info()

In [ ]:
ALL.shape

### 補足缺的時間點，和刪除重複或小於時間間隔的資料，並從 DCS#LINE and DCS#TAGCD 切出 116 個 features
總共的 features 可以切成 7 大類，116個

In [ ]:
ALL_fill_missing_time = pd.DataFrame()
for name in ALL['DCS#LINE'].unique():
    tmp = ALL[ALL['DCS#LINE']== name] #分出7大類
    #print(name)
    for tag in tmp['DCS#TAGCD'].unique(): #對每一類的 features做處理
        print(tag)
        tmp1 = tmp[tmp['DCS#TAGCD']== tag][['DCS#VALUE', 'datetime']]
        tmp1.reset_index(inplace= True, drop= True)
        start_end_time = pd.DataFrame(data={                # 設定資料開始跟結束的時間
                        'DCS#VALUE' : [np.nan, np.nan],
                        'datetime' : ['2021-08-01 00:00', '2022-07-31 23:55']
                })
        start_end_time = pd.to_datetime(start_end_time['datetime'], format= '%Y-%m-%d %H:%M').to_frame()
        tmp1 = pd.concat([start_end_time, tmp1], axis= 0, ignore_index= True)
        tmp1 = tmp1.sort_values(by= 'datetime').reset_index(drop= True)
        
        missing_time = pd.DataFrame(columns= ['Differ', 'previous time', 'least time']) #儲存缺的時間上下兩個時間點
        redundant_time = pd.DataFrame(columns= ['t', 't-1']) #儲存重複或小於時間間隔上下兩個時間點
        insert_time = pd.DataFrame(columns= ['DCS#VALUE', 'datetime']) # 補足缺失的時間點
        
        tmp2 = tmp1
        for i in range(1, len(tmp1)):
            diff = tmp1['datetime'][i] - tmp1['datetime'][i-1]
            if diff < timedelta(minutes= 5): #小於時間間隔的日期
                redundant_time = redundant_time.append({'t': tmp1['datetime'][i], 't-1': tmp1['datetime'][i-1]}, ignore_index= True)
                tmp2 = tmp1.drop(index= i)
            
            elif diff > timedelta(minutes= 5): #大於時間間隔的日期，代表中間有缺時間
                missing_time = missing_time.append({'Differ': diff, 'previous time': tmp1['datetime'][i-1], 'least time': tmp1['datetime'][i]}, ignore_index= True)
                #display(missing_time)
                insert = tmp1['datetime'][i-1]
                
                while diff > timedelta(minutes= 5): #補缺的時間，正確的間隔是 5 分鐘，所以每次補一個時間點後 diff 會減少 5 分鐘
                    insert += timedelta(minutes= 5)
                    diff -= timedelta(minutes= 5)
                    #print(f'diff: {diff}, insert: {insert}')
                    time = pd.DataFrame({'datetime': [insert], 'DCS#VALUE': [np.nan]})
                    insert_time = insert_time.append(time, ignore_index= True)
        
        tmp3 = pd.concat([insert_time, tmp2], ignore_index= True)
        tmp3 = tmp3.sort_values(by= 'datetime').reset_index(drop= True)
        tmp3 = tmp3.drop_duplicates(subset= ['datetime'], ignore_index= True)
        ALL_fill_missing_time = pd.concat([ALL_fill_missing_time, tmp3], axis= 1)
        ALL_fill_missing_time.rename(columns= {'DCS#VALUE' : str(name + '_' + tag), 'datetime' : str(tag + '_time')}, inplace= True)
        #display(insert_time)
#只留一個時間
time = ALL_fill_missing_time.iloc[:, 1].rename('datetime')
ALL_fill_missing_time = ALL_fill_missing_time.iloc[:, [i%2==0 for i in range(0, len(new.columns))]] #奇數 columns 是放 values，偶數 columns 是放 datetime
ALL_fill_missing_time = pd.concat([ALL_fill_missing_time, time], axis= 1)
ALL_fill_missing_time.shape

### 製造 Label，和先矯正對 label 不合理的資料
TotalActivePower 是累積發電量，所以要自己扣上個時間點，才會是這段時間的發電量，WF01_COUNTER_TotalActivePowerGen 也是同理
因為風機本身也需要電力，所以相減後可能會出現負值，因為是預測發電量，所以將負值都視為沒發電，用0取代

In [ ]:
diff1 = ALL_fill_missing_time['WF01_COUNTER_TotalActivePower']
diff1 = diff1.rename('WF01_ActivePower')
diff1 = diff1.diff()
ALL_fill_missing_time = pd.concat([ALL_fill_missing_time, diff1], axis=1)

diff1 = ALL_fill_missing_time['WF01_COUNTER_TotalActivePowerGen']
diff1 = diff1.rename('WF01_TotalActivePowerGen')
diff1 = diff1.diff()
ALL_fill_missing_time = pd.concat([ALL_fill_missing_time, diff1], axis=1)

In [ ]:
ALL_fill_missing_time.loc[ALL_fill_missing_time.WF01_ActivePower< 0, ['WF01_ActivePower']] = 0
ALL_fill_missing_time.loc[ALL_fill_missing_time.WF01_TotalActivePowerGen< 0, ['WF01_TotalActivePowerGen']] = 0
ALL_with_label = ALL_fill_missing_time.reset_index(drop= True)

### 使用移動平均補通訊異常值(-1000 and NaN)
移動平均 window=6，過去 30 分鐘的平均補， '2021-08-03 15:30:00' 前面多是通訊異常，所以先刪除再補

In [ ]:
ALL_with_label = ALL_with_label[ALL_with_label.datetime>= '2021-08-03 15:30:00'].reset_index(drop= True)
ALL_with_label.shape

In [ ]:
for label in ALL_with_label.columns.drop('datetime'):
    tmp = ALL_with_label[label].to_numpy()
    print(f'{label}')
    for i in range(6, len(new1)):
        if np.isnan(ALL_with_label[label][i]) or (ALL_with_label[label][i]== -1000):
            tmp[i] = np.mean(tmp[i- 6:i])
    ALL_with_label[label] = tmp

### 對 label 做 時間序列檢定，和畫 ACF and PACF 決定下一步要製造 lag幾期的 features對 label 做 時間序列檢定，和畫 ACF and PACF 決定下一步要製造 lag幾期的 features

In [ ]:
def adfuller_test(Power):
    result = adfuller(Power)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    for value, label in zip(result,labels):
        print(label+' : '+str(value) )
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")
        
adfuller_test(ALL_with_label['WF01_ActivePower'])

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax1 = fig.add_subplot(1, 1, 1)
fig = sm.graphics.tsa.plot_acf(ALL_with_label['WF01_ActivePower'], lags= 30, ax= ax1)
plt.xlabel('Time lag')
plt.ylim(0, 1)
plt.savefig('acf.png')
plt.show()

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax2 = fig.add_subplot(1, 1, 1)
fig = sm.graphics.tsa.plot_pacf(ALL_with_label['WF01_ActivePower'], lags= 10, ax= ax2)
plt.axhline(y= -1.96/np.sqrt(len(ALL_with_label['WF01_ActivePower'])), linestyle= '--', color= 'gray')
plt.axhline(y= 1.96/np.sqrt(len(ALL_with_label['WF01_ActivePower'])), linestyle= '--', color= 'gray')
plt.xlabel('Time lag')
plt.ylim(-0.25, 1)
plt.savefig('pacf.png')
plt.show()

### 製造出 t-1 ~ t-7，的 features 並移除了 t 時點的 features

In [ ]:
def make_t_mu_features(t):
    #new_with_t = new.iloc[t:, :].reset_index(drop= True)
    tmp2 = ALL_with_label.loc[t:, ['datetime','WF01_ActivePower']].reset_index(drop= True) #先把 label 拿出ㄓㄞ
    for col in ALL_with_label.drop('datetime', axis= 1).columns:
        for i in np.arange(1, t+1): #製造 t-1~t-7 的 features
            tmp = ALL_with_label.loc[t-i:new_with_t.shape[0]+(t-i-1), col].reset_index(drop= True)
            tmp.name = col + '_t-' + str(i)
            print(col + '_t-' + str(i))
            tmp2 = pd.concat([tmp2, tmp], axis= 1)
    return tmp2

In [ ]:
ALL_with_past_features = make_t_mu_features(7)
ALL_with_past_features.shape

### 刪除異常的 WF01_ActivePower and WF01_TotalActivePowerGen
每支風機裝置容量是 3600kWh，也就是一支風機每小時最多發 3600kWh，每 5 分鐘最大發電量就是 600 kWh

In [ ]:
ALL_with_past_features = ALL_with_past_features.drop(new_with_t[new_with_t['WF01_ActivePower']> 600].index)
for i in range(1, 8):
    name = 'WF01_ActivePower_t-' + str(i)
    new_with_t = ALL_with_past_features.drop(new_with_t[new_with_t[name]> 600].index)

In [ ]:
for i in range(1, 8):
    name = 'WF01_TotalActivePowerGen_t-' + str(i)
    ALL_with_past_features = ALL_with_past_features.drop(new_with_t[new_with_t[name]> 600].index)

### 選取和 label 相關係數前 20 高的當成 model 的 features

In [ ]:
#ALL_with_past_features = pd.read_csv('../wind power prediction/WF01_check_point.csv')

In [ ]:
corr_power = np.abs(ALL_with_past_features.corr()['WF01_ActivePower']).sort_values(ascending= False)

In [ ]:
feature_name = corr_power[0:21]
display(corr_power[0:21])
feature_name = feature_name.append(pd.Series(data= {'datetime':0})).index.tolist()
data = ALL_with_past_features[feature_name]
data.shape

In [ ]:
seed = 719
lasso = LassoCV(cv= 10, random_state= seed, max_iter= 100000, n_jobs= -1, eps= 1e-3)
lasso.fit(data.drop(['datetime','WF01_ActivePower'], axis= 1), data['WF01_ActivePower'])
keep_cols = [feature for feature, weight in zip(data.drop('WF01_ActivePower', axis= 1).columns, lasso.coef_) if weight != 0]
keep_cols

### 針對上一步所選取的 features 做 EDA針對上一步所選取的 features 做 EDA

In [ ]:
data.describe()

In [ ]:
sns.boxplot(x= data['WF01_ActivePower'])

In [ ]:
sns.histplot(data['WF01_ActivePower'])

In [ ]:
data.loc[data.WF01_ActivePower>600, ['WF01_ActivePower']].value_counts()

In [ ]:
sns.histplot(data[data.WF01_ActivePower<= 600]['WF01_ActivePower'])

In [ ]:
sns.histplot(data[data.WF01_ActivePower>600]['WF01_ActivePower'])

In [ ]:
sns.boxplot(x= data['WF01_ActivePower_t-2'])

In [ ]:
data['WF01_ActivePower_t-2'].value_counts()

In [ ]:
sns.histplot(x= data['WF01_ActivePower_t-2'])

In [ ]:
sns.histplot(data[data['WF01_ActivePower_t-2']<=600]['WF01_ActivePower_t-2'])

In [ ]:
sns.histplot(data[data['WF01_ActivePower_t-2']>600]['WF01_ActivePower_t-2'])

In [ ]:
sns.scatterplot(x= data['WF01_ActivePower_t-2'], y= data['WF01_ActivePower'])

In [ ]:
sns.boxplot(x= data['WF01_TotalActivePowerGen_t-2'])

In [ ]:
data['WF01_TotalActivePowerGen_t-2'].value_counts()

In [ ]:
sns.histplot(x= data['WF01_TotalActivePowerGen_t-2'])

In [ ]:
sns.histplot(x= data[data['WF01_TotalActivePowerGen_t-2']>0]['WF01_TotalActivePowerGen_t-2'])

In [ ]:
sns.scatterplot(x= data['WF01_TotalActivePowerGen_t-2'], y= data['WF01_ActivePower'])

In [ ]:
sns.boxplot(x= data['WF01_GRID_PossiblePower_t-3'])

In [ ]:
data['WF01_GRID_PossiblePower_t-3'].value_counts()

In [ ]:
sns.histplot(x= data['WF01_GRID_PossiblePower_t-3'])

In [ ]:
data[data['WF01_GRID_PossiblePower_t-3']<0]['WF01_GRID_PossiblePower_t-3']

In [ ]:
sns.histplot(x= data[data['WF01_GRID_PossiblePower_t-3']<0]['WF01_GRID_PossiblePower_t-3'])

In [ ]:
sns.scatterplot(x= data['WF01_GRID_PossiblePower_t-3'], y= data['WF01_ActivePower'])

In [ ]:
sns.boxplot(x= data['WF01_GRID_Current_P3_t-3'])

In [ ]:
data['WF01_GRID_Current_P3_t-3'].value_counts()

In [ ]:
sns.histplot(x= data['WF01_GRID_Current_P3_t-3'])

In [ ]:
sns.scatterplot(x= data['WF01_GRID_Current_P3_t-3'], y= data['WF01_ActivePower'])

In [ ]:
sns.boxplot(x= data['WF01_GRID_Current_P2_t-3'])

In [ ]:
data['WF01_GRID_Current_P2_t-3'].value_counts()

In [ ]:
sns.histplot(x= data['WF01_GRID_Current_P2_t-3'])

In [ ]:
sns.scatterplot(x= data['WF01_GRID_Current_P2_t-3'], y= data['WF01_ActivePower'])

In [ ]:
sns.boxplot(x= data['WF01_GRID_Power_t-3'])

In [ ]:
data['WF01_GRID_Power_t-3'].value_counts()

In [ ]:
sns.histplot(x= data['WF01_GRID_Power_t-3'])

In [ ]:
sns.scatterplot(x= data['WF01_GRID_Power_t-3'], y= data['WF01_ActivePower'])

In [ ]:
sns.boxplot(x= new_with_t['WF01_MAIN_WindSpeed_t-1'])

In [ ]:
plt.figure(figsize= (10, 8))
sns.histplot(x= new_with_t['WF01_COUNTER_TotalActivePower_t-1'])
plt.savefig('../wind power prediction/hist_totalAcitvePower.png')
plt.show()

In [ ]:
plt.figure(figsize= (10, 8))
sns.histplot(x= new_with_t['WF01_ActivePower'])
plt.savefig('../wind power prediction/hist_ActivePower.png')
plt.show()

### 使用 20210803~20220630 當成 train set，20220731 當成 test set

In [ ]:
# 按時間月份切，2022-07 當 test set
train = data.loc[(data.datetime<= '2022-05-31 23:55:00')] #不能用 06-31，因為6月只有30天
test = data.loc[data.datetime >= '2022-07-01 00:00:00']
test_time = test['datetime']
x_train, y_train = train.drop(['WF01_ActivePower', 'datetime'], axis= 1).to_numpy(), train['WF01_ActivePower'].to_numpy()
x_test, y_test = test.drop(['WF01_ActivePower', 'datetime'], axis= 1).to_numpy(), test['WF01_ActivePower'].to_numpy()

In [ ]:
#shuffle False
x_train, x_test, y_train, y_test = train_test_split(data.drop(['WF01_ActivePower', 'datetime'], axis= 1).to_numpy(), data['WF01_ActivePower'].to_numpy(), test_size= 0.2, shuffle= False, random_state= seed)

In [ ]:
#shuffle True
x_train, x_test, y_train, y_test = train_test_split(data.drop(['WF01_ActivePower', 'datetime'], axis= 1).to_numpy(), data['WF01_ActivePower'].to_numpy(), test_size= 0.2, shuffle= True, random_state= seed)

In [ ]:
print(y_train.shape)
print(y_test.shape)

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

### 訓練模型

In [ ]:
def smape(y, pred):
    smape = np.empty(len(y))
    for i in range(len(y)):
        if pred[i] + y[i]== 0:
            smape[i] = np.abs(pred[i] - y[i]) / 0.5
        else:
            smape[i] = np.abs(pred[i] - y[i]) / ((np.abs(pred[i]) + np.abs(y[i])) * 0.5)
    return np.mean(smape)

In [ ]:
def mape(y, pred):
    mape = np.empty(len(y))
    for i in range(len(y)):
        if y[i] == 0:
            mape[i] = np.abs((pred[i] - y[i]) / 1)
        else:
            mape[i] = np.abs((pred[i] - y[i]) / y[i])
    return np.mean(mape)

In [ ]:
mape_score = make_scorer(mape, greater_is_better= False)

In [ ]:
def print_score(self, x_train, x_test, y_train, y_test):
    train_pred = self.predict(x_train)
    test_pred = self.predict(x_test)
    train_pred[train_pred<0] = 0
    train_pred[train_pred>600] = 600
    test_pred[test_pred<0] = 0
    test_pred[test_pred>600] = 600
    
    
    train_mse = mean_squared_error(y_train, train_pred)
    test_mse = mean_squared_error(y_test, test_pred)
    
    train_mae = mean_absolute_error(y_train, train_pred)
    test_mae = mean_absolute_error(y_test, test_pred)
    
    train_mape = mape(y_train, train_pred)
    test_mape = mape(y_test, test_pred)
    
    
    train_smape = smape(y_train, train_pred)
    test_smape = smape(y_test, test_pred)
    
    train_r2 = r2_score(y_train, train_pred)
    test_r2 = r2_score(y_test, test_pred)
    
    cross_mape = cross_val_score(self, x_train, y_train, cv= 10, scoring= mape_score)
    
    print(f'train/test MSE {train_mse:.9f}/{test_mse:.9f}')
    print(f'train/test MAE {train_mae:.9f}/{test_mae:.9f}')
    print(f'train/test MAPE {train_mape:.9f}/{test_mape:.9f}')    
    print(f'train/test SMAPE {train_smape:.9f}/{test_smape:.9f}')
    print(f'train/test r2_squared {train_r2:.9f}/{test_r2:.9f}\n')
    print(f'MAPE for cv= 10: {-cross_mape.mean():.9f} (+/- {cross_mape.std():.9f})')

In [ ]:
def predict_plot(self, x_test, y_test, test_time):
    test_pred = self.predict(x_test)
    test_pred[test_pred<0] = 0
    test_pred[test_pred>600] = 600
    # test plot
    plt.figure(figsize= (20, 8))
    plt.title('test predicted V.S. Actual')
    plt.plot(test_time, y_test, color= 'C0', marker= 'o', label= 'Actual Power')
    plt.plot(test_time, test_pred, color= 'C1', marker= 'o', label= 'Predicted Power')
    plt.legend(prop= {'size': 10})
    plt.savefig('../wind power prediction/predict_plot.png')
    plt.show()

In [ ]:
def shuffle_predict_plot(self, x_test, y_test, test_time):
    test_pred = self.predict(x_test)
    test_pred[test_pred<0] = 0
    test_pred[test_pred>600] = 600
    # test plot
    x = range(len(y_test))
    plt.figure(figsize= (15, 8))
    plt.title('Predicted V.S. Actual')
    plt.plot(x, y_test, color= 'C0', marker= 'o', label= 'Actual Power')
    plt.plot(x, test_pred, color= 'C1', marker= 'o', label= 'Predicted Power')
    plt.legend(prop= {'size': 10})
    plt.show()

In [ ]:
seed = 719

In [ ]:
# 按時間月份切，2022-07 當 test set
xgb = XGBRegressor(learning_rate= 0.1, n_estimators= 100, max_depth= 10, objective= 'reg:squarederror', n_jobs= 20, seed= seed)
xgb.fit(x_train, y_train)
print_score(xgb, x_train, x_test, y_train, y_test)
predict_plot(xgb, x_test, y_test, test_time)

In [ ]:
# test size= 0.2, shuffle= True
xgb = XGBRegressor(learning_rate= 0.1, n_estimators= 100, max_depth= 10, objective= 'reg:squarederror', n_jobs= 20, seed= seed)
xgb.fit(x_train, y_train)
print_score(xgb, x_train, x_test, y_train, y_test)
shuffle_predict_plot(xgb, x_test, y_test, test_time)

In [ ]:
#切一天來畫圖
plot_data = data[(data.datetime>= '2022-07-21 00:00:00')&(data.datetime<= '2022-07-21 23:55:00')]
plot_x, plot_y = plot_data.drop(['WF01_ActivePower', 'datetime'], axis= 1).to_numpy(), plot_data['WF01_ActivePower'].to_numpy()
plot_time = plot_data['datetime']
#plot_time = pd.to_datetime(plot_time, format= '%Y-%m-%d %H:%M:%S')
plot_x = scaler.transform(plot_x)
predict_plot(xgb, plot_x, plot_y, plot_time)

In [ ]:
# test size= 0.2, shuffle= False
xgb = XGBRegressor(learning_rate= 0.1, n_estimators= 100, max_depth= 10, objective= 'reg:squarederror', n_jobs= 20, seed= seed)
xgb.fit(x_train, y_train)
print_score(xgb, x_train, x_test, y_train, y_test)
shuffle_predict_plot(xgb, x_test, y_test, test_time)

In [ ]:
# 按時間月份切，2022-07 當 test set
rfreg = RandomForestRegressor(n_estimators= 100, criterion= 'squared_error', max_depth= 10, n_jobs= 10, random_state= seed)
rfreg.fit(x_train, y_train)
print_score(rfreg, x_train, x_test, y_train, y_test)
predict_plot(rfreg, x_test, y_test, test_time)

In [ ]:
#test size= 0.2, shuffle= True
rfreg = RandomForestRegressor(n_estimators= 100, criterion= 'squared_error', max_depth= 10, n_jobs= 10, random_state= seed)
rfreg.fit(x_train, y_train)
print_score(rfreg, x_train, x_test, y_train, y_test)
shuffle_predict_plot(rfreg, x_test, y_test, test_time)

In [ ]:
predict_plot(rfreg, plot_x, plot_y, plot_time)

In [ ]:
#test size= 0.2, shuffle= False
rfreg = RandomForestRegressor(n_estimators= 100, criterion= 'squared_error', max_depth= 10, n_jobs= 10, random_state= seed)
rfreg.fit(x_train, y_train)
print_score(rfreg, x_train, x_test, y_train, y_test)
shuffle_predict_plot(rfreg, x_test, y_test, test_time)

In [ ]:
# 按時間月份切，2022-07 當 test set
svr = SVR(kernel= 'rbf', gamma= 'auto' ,C= 1, epsilon= 0.1, cache_size= 1000, max_iter= 50000)
svr.fit(x_train, y_train)
print_score(svr, x_train, x_test, y_train, y_test)
predict_plot(svr, x_test, y_test, test_time)

In [ ]:
#test size= 0.2, shuffle= True
svr = SVR(kernel= 'rbf', gamma= 'auto' ,C= 1, epsilon= 0.1, cache_size= 1000, max_iter= 50000)
svr.fit(x_train, y_train)
print_score(svr, x_train, x_test, y_train, y_test)
shuffle_predict_plot(xgb, x_test, y_test, test_time)

In [ ]:
predict_plot(svr, plot_x, plot_y, plot_time)

In [ ]:
#test size= 0.2, shuffle= False
svr = SVR(kernel= 'rbf', gamma= 'auto' ,C= 1, epsilon= 0.1, cache_size= 1000, max_iter= 50000)
svr.fit(x_train, y_train)
print_score(svr, x_train, x_test, y_train, y_test)
shuffle_predict_plot(xgb, x_test, y_test, test_time)

In [ ]:
# knnreg = KNeighborsRegressor(n_neighbors= 100, weights= 'uniform', algorithm= 'auto', leaf_size= 30, n_jobs= 1)
# knnreg.fit(x_train, y_train)
# print_score(knnreg, x_train, x_test, y_train, y_test)
# shuffle_predict_plot(svr, x_train, x_test, y_train, y_test, train_time, test_time)

In [ ]:
# 按時間月份切，2022-07 當 test set
ridge= Ridge(alpha= 0.5)
ridge.fit(x_train, y_train)
print_score(ridge, x_train, x_test, y_train, y_test)
predict_plot(ridge, x_test, y_test, test_time)

In [ ]:
#test size= 0.2, shuffle= True
ridge= Ridge(alpha= 0.5)
ridge.fit(x_train, y_train)
print_score(ridge, x_train, x_test, y_train, y_test)

In [ ]:
#test size= 0.2, shuffle= False
ridge= Ridge(alpha= 0.5)
ridge.fit(x_train, y_train)
print_score(ridge, x_train, x_test, y_train, y_test)

In [ ]:
# 按時間月份切，2022-07 當 test set
bayesreg = BayesianRidge(tol= 1e-3, alpha_1= 1e-6, alpha_2 = 1e-6, lambda_1= 1e-6, lambda_2= 1e-6)
bayesreg.fit(x_train, y_train)
print_score(bayesreg, x_train, x_test, y_train, y_test)
predict_plot(bayesreg, x_test, y_test, test_time)

In [ ]:
#test size= 0.2, shuffle= True
bayesreg = BayesianRidge(tol= 1e-3, alpha_1= 1e-6, alpha_2 = 1e-6, lambda_1= 1e-6, lambda_2= 1e-6)
bayesreg.fit(x_train, y_train)
print_score(bayesreg, x_train, x_test, y_train, y_test)
shuffle_predict_plot(bayesreg, x_test, y_test, test_time)

In [ ]:
#test size= 0.2, shuffle= False
bayesreg = BayesianRidge(tol= 1e-3, alpha_1= 1e-6, alpha_2 = 1e-6, lambda_1= 1e-6, lambda_2= 1e-6)
bayesreg.fit(x_train, y_train)
print_score(bayesreg, x_train, x_test, y_train, y_test)
shuffle_predict_plot(bayesreg, x_test, y_test, test_time)

In [ ]:
# 按時間月份切，2022-07 當 test set
dt = DecisionTreeRegressor(criterion= 'squared_error', max_depth= 10, random_state= seed)
dt.fit(x_train, y_train)
print_score(ridge, x_train, x_test, y_train, y_test)
shuffle_predict_plot(dt, x_test, y_test, test_time)

In [ ]:
#test size= 0.2, shuffle= True
dt = DecisionTreeRegressor(criterion= 'squared_error', max_depth= 10, random_state= seed)
dt.fit(x_train, y_train)
print_score(ridge, x_train, x_test, y_train, y_test)
shuffle_predict_plot(dt, x_test, y_test, test_time)

In [ ]:
#test size= 0.2, shuffle= False
dt = DecisionTreeRegressor(criterion= 'squared_error', max_depth= 10, random_state= seed)
dt.fit(x_train, y_train)
print_score(ridge, x_train, x_test, y_train, y_test)
shuffle_predict_plot(dt, x_test, y_test, test_time)

### Hyperparameter fine tuning for XGBRegressor
主要針對 learning_rate、n_estimators、max_depth、和 reg:pseudohubererror，做調整

#### RandomizedSearch

In [ ]:
distributions = {
                 'learning_rate' : uniform(loc= 0.1, scale= 0.9),
                 'n_estimators' : randint(100, 200),
                 'max_depth' : randint(10, 30),
                    }
mape_score = make_scorer(mape, greater_is_better= False)
xgb = XGBRegressor(objective= 'reg:squarederror', seed= seed)
xgb_search = RandomizedSearchCV(xgb, param_distributions= distributions, cv= 10, scoring= mape_score, n_jobs= 10, random_state= seed)
search = xgb_search.fit(x_train, y_train)

In [ ]:
search.best_estimator_

In [ ]:
search.cv_results_

In [ ]:
rfreg = RandomForestRegressor(n_estimators= 100, criterion= 'squared_error', max_depth= 10, n_jobs= 20, random_state= seed)
rfreg.fit(x_train, y_train)
print_score(rmreg, x_train, x_test, y_train, y_test)
#predict_plot(rmreg, x_train, x_test, y_train, y_test, train_time, test_time)

In [ ]:
distributions = {
                    'n_estimators' : randint(100, 500),
                    'criterion' : ['squared_error', 'absolute_error', 'poisson'],
                    'max_depth' : randint(10, 50),
}
rfreg = RandomForestRegressor(random_state= seed)
rfreg_search = RandomizedSearchCV(rfreg, param_distributions= distributions, cv= 10, scoring= mape_score, n_jobs= 10, random_state= seed)
search = rfreg_search.fit(x_train, y_train)

In [ ]:
search.best_estimator_

In [ ]:
search.cv_results_

#### GridSearch

In [ ]:
param_grid = {
            'learning_rate' : [0.27, 0.28, 0.29],
            'n_estimators' : [181, 182, 183],
            'max_depth' : [19, 20, 21]
}

grid_search = GridSearchCV(xgb, param_grid, cv= 10, n_jobs= 10, scoring= mape_score)
search = grid_search.fit(x_train, y_train)

In [ ]:
search.best_estimator_

In [ ]:
search.cv_results_

In [ ]:
xgb = XGBRegressor(learning_rate= 0.28, n_estimators= 182, max_depth= 10, objective= 'reg:squarederror', n_jobs= 20, seed= seed)
xgb.fit(x_train, y_train)
print_score(xgb, x_train, x_test, y_train, y_test)
shuffle_predict_plot(xgb, x_test, y_test, test_time)

### Ensemble learning
使用 MAPE 當成每個預測結果的權重

In [ ]:
xgb_bag = BaggingRegressor(base_estimator= XGBRegressor(learning_rate= 0.28, n_estimators= 10, max_depth= 10, objective= 'reg:squarederror', n_jobs= 20, seed= seed), n_estimators= 100, max_features= 0.2, bootstrap= True, n_jobs= 10)
xgb_bag.fit(x_train, y_train)
print_score(xgb_bag, x_train, x_test, y_train, y_test)
shuffle_predict_plot(xgb_bag, x_test, y_test, test_time)

In [ ]:
predict_plot(xgb_bag, plot_x, plot_y, plot_time)

In [ ]:
svr_bag = BaggingRegressor(base_estimator= SVR(kernel= 'rbf', gamma= 'auto' ,C= 1, epsilon= 0.1, cache_size= 1000, max_iter= 50000), n_estimators= 100, max_features= 0.2, bootstrap= True, n_jobs= 20)
svr_bag.fit(x_train, y_train)
print_score(svr_bag, x_train, x_test, y_train, y_test)
shuffle_predict_plot(svr_bag, x_test, y_test, test_time)

In [ ]:
rf_bag = BaggingRegressor(base_estimator= RandomForestRegressor(criterion= 'squared_error', max_depth= 10, n_jobs= 20, random_state= seed), n_estimators= 100, max_features= 0.2, bootstrap= True, n_jobs= 10)
rf_bag.fit(x_train, y_train)
print_score(rf_bag, x_train, x_test, y_train, y_test)
shuffle_predict_plot(rf_bag, x_test, y_test, test_time)

In [ ]:
predict_plot(rf_bag, plot_x, plot_y, plot_time)

In [ ]:
# ensemble average
estimators = [
    ('xgb', xgb),
    ('rfreg', rfreg)
]
reg = VotingRegressor(estimators= estimators, n_jobs= 10)
reg.fit(x_train, y_train)
print_score(reg, x_train, x_test, y_train, y_test)
shuffle_predict_plot(reg, x_test, y_test, test_time)

In [ ]:
predict_plot(reg, plot_x, plot_y, plot_time)

In [ ]:
def ensemble_weight(model1, model2, x, y):
    pred1 = model1.predict(x)
    pred2 = model2.predict(x)
    w1 = 1 / mape(y, pred1)
    w2 = 1 / mape(y, pred2)
    return w1, w2

In [ ]:
def ensemble_output(model1, model2, x, w1, w2):
    pred1 = model1.predict(x)
    pred2 = model2.predict(x)
    return (w1 /( w1 + w2)) * pred1 + (w2/ (w1 + w2)) * pred2

In [ ]:
# ensemble weighted average
w1, w2 = ensemble_weight(xgb, rfreg, x_train, y_train)
train_ensemble = ensemble_output(xgb, rfreg, x_train, w1, w2)
test_ensemble = ensemble_output(xgb, rfreg, x_test, w1, w2)

train_mse = mean_squared_error(y_train, train_ensemble)
test_mse = mean_squared_error(y_test, test_ensemble)

train_mae = mean_absolute_error(y_train, train_ensemble)
test_mae = mean_absolute_error(y_test, test_ensemble)

train_mape = mape(y_train, train_ensemble)
test_mape = mape(y_test, test_ensemble)

train_smape = smape(y_train, train_ensemble)
test_smape = smape(y_test, test_ensemble)

train_r2 = r2_score(y_train, train_ensemble)
test_r2 = r2_score(y_test, test_ensemble)

print(f'train/test MSE {train_mse:.9f}/{test_mse:.9f}')
print(f'train/test MAE {train_mae:.9f}/{test_mae:.9f}')
print(f'train/test MAPE {train_mape:.9f}/{test_mape:.9f}')    
print(f'train/test SMAPE {train_smape:.9f}/{test_smape:.9f}')
print(f'train/test r2_squared {train_r2:.9f}/{test_r2:.9f}\n')

In [ ]:
# ensemble by MLP
estimators = [
    ('xgb', xgb),
    ('rfreg', rfreg)
]
y_train = np.ravel(y_train)
clf = StackingRegressor(estimators= estimators, final_estimator= MLPRegressor(activation= 'relu', alpha= 0.1, hidden_layer_sizes= (4,2), learning_rate= 'constant', learning_rate_init= 1e-3, solver= 'adam', max_iter= 2000, random_state= seed), n_jobs=10)
clf.fit(x_train, y_train)

print_score(clf, x_train, x_test, y_train, y_test)
shuffle_predict_plot(clf, x_test, y_test, test_time)

In [ ]:
predict_plot(clf, plot_x, plot_y, plot_time)

In [ ]:
# ensemble average xgb and svr
estimators = [
    ('xgb', xgb),
    ('svr', svr)
]
reg = VotingRegressor(estimators= estimators, n_jobs= 10)
reg.fit(x_train, y_train)
print_score(reg, x_train, x_test, y_train, y_test)
shuffle_predict_plot(reg, x_test, y_test, test_time)

In [ ]:
w1, w2 = ensemble_weight(xgb, svr, x_train, y_train)
train_ensemble = ensemble_output(xgb, svr, x_train, w1, w2)
test_ensemble = ensemble_output(xgb, svr, x_test, w1, w2)

train_mse = mean_squared_error(y_train, train_ensemble)
test_mse = mean_squared_error(y_test, test_ensemble)

train_mae = mean_absolute_error(y_train, train_ensemble)
test_mae = mean_absolute_error(y_test, test_ensemble)

train_mape = mape(y_train, train_ensemble)
test_mape = mape(y_test, test_ensemble)

train_smape = smape(y_train, train_ensemble)
test_smape = smape(y_test, test_ensemble)

train_r2 = r2_score(y_train, train_ensemble)
test_r2 = r2_score(y_test, test_ensemble)

print(f'train/test MSE {train_mse:.9f}/{test_mse:.9f}')
print(f'train/test MAE {train_mae:.9f}/{test_mae:.9f}')
print(f'train/test MAPE {train_mape:.9f}/{test_mape:.9f}')    
print(f'train/test SMAPE {train_smape:.9f}/{test_smape:.9f}')
print(f'train/test r2_squared {train_r2:.9f}/{test_r2:.9f}\n')

In [ ]:
# ensemble by MLP
estimators = [
    ('xgb', xgb),
    ('rfreg', rfreg)
]
y_train = np.ravel(y_train)
clf = StackingRegressor(estimators= estimators, final_estimator= MLPRegressor(activation= 'relu', alpha= 0.1, hidden_layer_sizes= (4,2), learning_rate= 'constant', learning_rate_init= 1e-3, solver= 'adam', max_iter= 2000, random_state= seed), n_jobs=10)
clf.fit(x_train, y_train)

print_score(clf, x_train, x_test, y_train, y_test)
shuffle_predict_plot(clf, x_test, y_test, test_time)

In [ ]:
#ensemble average xgb and bayesreg
estimators = [
    ('xgb', xgb),
    ('bayesreg', bayesreg)
]
reg = VotingRegressor(estimators= estimators, n_jobs= 10)
reg.fit(x_train, y_train)
print_score(reg, x_train, x_test, y_train, y_test)
shuffle_predict_plot(reg, x_test, y_test, test_time)

In [ ]:
w1, w2 = ensemble_weight(xgb, svr, x_train, y_train)
train_ensemble = ensemble_output(xgb, bayesreg, x_train, w1, w2)
test_ensemble = ensemble_output(xgb, bayesreg, x_test, w1, w2)

train_mse = mean_squared_error(y_train, train_ensemble)
test_mse = mean_squared_error(y_test, test_ensemble)

train_mae = mean_absolute_error(y_train, train_ensemble)
test_mae = mean_absolute_error(y_test, test_ensemble)

train_mape = mape(y_train, train_ensemble)
test_mape = mape(y_test, test_ensemble)

train_smape = smape(y_train, train_ensemble)
test_smape = smape(y_test, test_ensemble)

train_r2 = r2_score(y_train, train_ensemble)
test_r2 = r2_score(y_test, test_ensemble)

print(f'train/test MSE {train_mse:.9f}/{test_mse:.9f}')
print(f'train/test MAE {train_mae:.9f}/{test_mae:.9f}')
print(f'train/test MAPE {train_mape:.9f}/{test_mape:.9f}')    
print(f'train/test SMAPE {train_smape:.9f}/{test_smape:.9f}')
print(f'train/test r2_squared {train_r2:.9f}/{test_r2:.9f}\n')

In [ ]:
# ensemble by MLP
estimators = [
    ('xgb', xgb),
    ('bayesreg', bayesreg)
]
y_train = np.ravel(y_train)
clf = StackingRegressor(estimators= estimators, final_estimator= MLPRegressor(activation= 'relu', alpha= 0.1, hidden_layer_sizes= (4,2), learning_rate= 'constant', learning_rate_init= 1e-3, solver= 'adam', max_iter= 2000, random_state= seed), n_jobs=10)
clf.fit(x_train, y_train)

print_score(clf, x_train, x_test, y_train, y_test)
shuffle_predict_plot(clf, x_test, y_test, test_time)